In [2]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle


/Users/annafilina/ds_bootcamp/Personal-Shopper/.PerSho/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cpu')
model_name = "cointegrated/rubert-tiny2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### Загружаю датасет, Обучаю эмбединг

In [4]:
def_new = pd.read_csv('../Personal-Shopper/all+++.csv') #../Personal-Shopper/csv/merged.csv
annot_new = def_new['annotation']
def_new.shape



(32214, 6)

In [12]:
def_new.columns

Index(['page_url', 'image_url', 'author', 'title', 'annotation', 'genre',
       'random_category'],
      dtype='object')

In [5]:
import random #для теста по подбору категорий

categories = ['up', 'bottom', 'full', 'shoes', 'bag', 'acc', 'outwear']
def_new['random_category'] = random.choices(categories, k=len(def_new))


In [44]:
unique_categories = def_new[['cat_1', 'cat_2', 'cat_3']].drop_duplicates() 
#когда датасет будет готов надо будет проставитть категории up, bottom, shoes, bags and acc
unique_categories


,cat_1,cat_2,cat_3
0,Одежда,Блузки_Рубашки,Рубашки
38,Одежда,Блузки_Рубашки,Блузы
76,Одежда,Блузки_Рубашки,Повседневные
114,Одежда,Блузки_Рубашки,Классические
152,Одежда,Блузки_Рубашки,Unisex
154,Одежда,Костюмы,Костюмы
165,Одежда,Костюмы,Однобортные
212,Одежда,Костюмы,Двубортные
259,Одежда,Костюмы,Жилеты
306,Одежда,Denim,Джинсы


In [46]:
unique_categories.to_csv('unique_categories.csv', index=False) 


In [6]:
#средняя размерность
average_length = annot_new.apply(lambda x: len(str(x))).mean()
average_length


762.9917116781523

In [7]:
annot_new

0        "Крестоносцы" — одно из лучших произведений по...
1        Роман «Игрок» — не просто одно из всемирно изв...
2        «„Горе от ума“ появилось раньше Онегина, Печор...
3        В нашем новом проекте вы найдете как книги из ...
4        История развития литературы – понятие куда бол...
                               ...                        
32209    Умение различать геометрические фигуры, выбира...
32210    В книге отражается развитие ребенка от рождени...
32211    Это практичное руководство дает "подсказки и с...
32212    Книга об эмоциях в картинках и ситуациях, поня...
32213    Книга об эмоциях в картинках и ситуациях, поня...
Name: annotation, Length: 32214, dtype: object

In [55]:
embeddings_256 = []

embeddings = [embeddings_256] 
for i in range(0,len(embeddings)):
    
    if i == 0:
        length = 256
    embedding =embeddings[i]
    for annotation in tqdm(annot_new):
        annotation_tokens = tokenizer.encode_plus(
            annotation,
            add_special_tokens=True,
            max_length=length, # Ограничение на максимальную длину входной последовательности
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины если слов меньше заданной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )

        with torch.no_grad():
            # Получим эмбеддинг аннотации
            outputs = model(**annotation_tokens)
            hidden_states = outputs.hidden_states
            last_hidden_state = hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            last_hidden_state = torch.nn.functional.normalize(last_hidden_state)

            embedding.append((last_hidden_state).squeeze())

  0%|          | 0/32214 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/annafilina/ds_bootcamp/Personal-Shopper/.PerSho/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|█████████

#### Смотрю рекомендации 

In [8]:
from collections import defaultdict

In [69]:
with open('/Users/annafilina/ds_bootcamp/Personal-Shopper/embeddings_256.pkl', 'wb') as f:
    pickle.dump(embeddings_256, f)

In [9]:
with open("embeddings_256.pkl", "rb") as f:
    embeddings_256 = pickle.load(f)

In [13]:
queries = ['книга про путешествие ребенка в космосе']
length = 256
embeddings = embeddings_256
top_elements_per_query = defaultdict(lambda: defaultdict(list))

for query in queries:
    
    for category in ['up', 'bottom', 'full', 'shoes', 'bag', 'acc']:
        query_tokens = tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=length,
            pad_to_max_length=True,
            return_tensors='pt'
        )

        with torch.no_grad():
            query_outputs = model(**query_tokens)
            query_hidden_states = query_outputs.hidden_states
            query_last_hidden_state = query_hidden_states[-1][:, 0, :]
            query_embedding = query_last_hidden_state.squeeze()
            cosine_similarities = torch.nn.functional.cosine_similarity(
                query_embedding.unsqueeze(0),
                torch.stack(embeddings)
            )

            cosine_similarities = cosine_similarities.numpy()

            indices = np.argsort(cosine_similarities)[::-1]

            elements = []
            for i in indices:
                if def_new['random_category'][i] == category:
                    url = def_new['page_url'][i]
                    probability = cosine_similarities[i]
                    elements.append((url, probability))
                    if len(elements) == 5:
                        break

            top_elements_per_query[query][category] = elements


for query, elements_per_category in top_elements_per_query.items():
    print(f'\n{query}')
    for category, elements in elements_per_category.items():
        print(f'{category}:')
        print(f'{len(elements)} elements')
        for url, probability in elements:
            print(f'{url}: {probability:.4f}')


/Users/annafilina/ds_bootcamp/Personal-Shopper/.PerSho/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(



книга про путешествие ребенка в космосе
up:
5 elements
https://www.biblio-globus.ru/product/10392425: 0.6425
https://www.biblio-globus.ru/product/10765532: 0.6374
https://www.biblio-globus.ru/product/10909038: 0.6274
https://www.biblio-globus.ru/product/10727203: 0.6247
https://www.biblio-globus.ru/product/10864201: 0.6153
bottom:
5 elements
https://www.biblio-globus.ru/product/10869093: 0.6492
https://www.biblio-globus.ru/product/10895226: 0.6176
https://www.biblio-globus.ru/product/10865061: 0.6173
https://www.biblio-globus.ru/product/10888881: 0.6149
https://www.biblio-globus.ru/product/10407041: 0.6100
full:
5 elements
https://www.biblio-globus.ru/product/10659262: 0.6773
https://www.biblio-globus.ru/product/10625211: 0.6445
https://www.biblio-globus.ru/product/10767746: 0.6343
https://www.biblio-globus.ru/product/10715484: 0.6264
https://www.biblio-globus.ru/product/10646646: 0.6191
shoes:
5 elements
https://www.biblio-globus.ru/product/10528868: 0.6556
https://www.biblio-globus.

In [25]:
temp_basket = []
for query, elements_per_category in top_elements_per_query.items():
    for category, elements in elements_per_category.items():
        for url, probability in elements:
            row = def_new.loc[def_new['page_url'] == url].iloc[0].tolist()
            row.append(probability)
            temp_basket.append(row)

columns = ['page_url', 'image_url', 'author', 'title', 'annotation', 'genre', 'random_category', 'probability'] #заменить на наши
temp_basket = pd.DataFrame(temp_basket, columns=columns)


In [26]:

temp_basket


,page_url,image_url,author,title,annotation,genre,random_category,probability
0,https://www.biblio-globus.ru/product/10392425,https://static1.bgshop.ru/imagehandler.ashx?wi...,Цхай И. В.,Грибное лукошко : сказка,Книга входит в серию детской познавательной ли...,Раскраски,up,0.642542
1,https://www.biblio-globus.ru/product/10765532,https://static1.bgshop.ru/imagehandler.ashx?wi...,Перри Л.,"Ракета стартует. История освоения космоса, кот...",История освоения космоса складывается не тольк...,Космогония. Космология,up,0.637418
2,https://www.biblio-globus.ru/product/10909038,https://static1.bgshop.ru/imagehandler.ashx?wi...,Малый Р.,Книга исчезнувших животных. BIObook А. Толмачёва,Откройте тайны животного мира с «BIObook А. То...,Естественные науки. Математика,up,0.627371
3,https://www.biblio-globus.ru/product/10727203,https://static1.bgshop.ru/imagehandler.ashx?wi...,Хейердал Т.,"Путешествие ""Кон-Тики""",Вы когда-нибудь переплывали океан? А на плоту ...,Страноведение. Регионоведение,up,0.624707
4,https://www.biblio-globus.ru/product/10864201,https://static1.bgshop.ru/imagehandler.ashx?wi...,Венедиктова Ю.,"Магия ""Чехарды""",Волшебство рядом! Однажды на чердаке собственн...,Российская фантастика и триллер для детей и юн...,up,0.615309
5,https://www.biblio-globus.ru/product/10869093,https://static1.bgshop.ru/imagehandler.ashx?wi...,Ганери А.,Роботы. От пылесоса до лунохода,Роботы повсюду! Они летают в космос и доставля...,Универсальные детские энциклопедии,bottom,0.649235
6,https://www.biblio-globus.ru/product/10895226,https://static1.bgshop.ru/imagehandler.ashx?wi...,Бусахин С,Мыс Доброй Надежды,"Мыс Доброй Надежды - романтический образ, прес...",Современная проза,bottom,0.617620
7,https://www.biblio-globus.ru/product/10865061,https://static1.bgshop.ru/imagehandler.ashx?wi...,МакРей Э.,Рептилии,Книга «Рептилии» из серии «Детская энциклопеди...,Универсальные детские энциклопедии,bottom,0.617323
8,https://www.biblio-globus.ru/product/10888881,https://static1.bgshop.ru/imagehandler.ashx?wi...,Дмитриева В.Г.,Увлекательные лабиринты по городам и странам,«Увлекательные лабиринты по городам и странам»...,Экскурсионно-туристическая и краеведческая работа,bottom,0.614910
9,https://www.biblio-globus.ru/product/10407041,https://static1.bgshop.ru/imagehandler.ashx?wi...,Мур Г.,На луне: Загадки и головоломки,Не паникуй! Ты всего лишь летишь на космическо...,Пословицы. Поговорки. Загадки,bottom,0.610047
